# Error Analysis

## Load Data

In [71]:
import pandas as pd
import numpy as np

TEST_DF = pd.read_csv("../data/test_data.csv")

baseline_predictions_file = "BERT-Predictions.npy"
best_extension_predictions_file = "Hard Vote-Predictions.npy"
ground_truth_file = "test_labels.npy"

BASELINE_PREDICTIONS = np.load(baseline_predictions_file)
BEST_EXTENSION_PREDICTIONS = np.load(best_extension_predictions_file)
GROUND_TRUTH = np.load(ground_truth_file)

# Add arrays as new columns
TEST_DF['baseline_predictions'] = BASELINE_PREDICTIONS
TEST_DF['best_extension_predictions'] = BEST_EXTENSION_PREDICTIONS
TEST_DF['ground_truth'] = GROUND_TRUTH

TEST_DF.head()

,label,text,baseline_predictions,best_extension_predictions,ground_truth
0,0,"NEWLINE_TOKEN*:Yes, indeed. A misunderstanding caused by my not being able to see deleted versions.",0,0,0
1,0,"I'm still fairly new at all of this, so if anyone has some advice on the things I've written, please feel free to let me know.",0,0,0
2,0,"stop what?! \n\nHow is it spam to post information relevant to an article and post a link in the talk page, as you suggested, which is also relevant to the article? I see links in Wikipedia articles all the time, but they don't get deleted neurotically.",0,0,0
3,0,"`NEWLINE_TOKEN:::::::Practice saying: ``You were right, I was wrong, I apologise and I will go away.`` `",0,0,0
4,0,NEWLINE_TOKEN I think its safe to say that its at least meant to be a tie in to Begins the way BTAS was a tie in to the Burton films. Shouldnt this be mentioned?NEWLINE_TOKEN,0,0,0


## Analyze Published Baseline Errors, Hard Voting Correct

In [72]:
subset_df = TEST_DF[(TEST_DF['ground_truth'] == TEST_DF['best_extension_predictions']) & (TEST_DF['ground_truth'] != TEST_DF['baseline_predictions'])].reset_index(drop = True)
subset_df.head()

,label,text,baseline_predictions,best_extension_predictions,ground_truth
0,0,"""What a load of codswallop, you tell me that when my ankle starts to itch in the middle of the night, I get pleasure out of rubbing it raw, whilst trying not to wake the Mrs? Another complete and utter waste of money, that could have been spent on medical research into life threatning illness's.""",1,0,0
1,1,"That's a deal breaker for me. I've always said if my car blows more trannies than I do, its not a good car.",0,1,1
2,0,@a_man_in_black and now mods of ghazi are under fire by a mob that appears to have been led by you. :(,1,0,0
3,1,Tell your bitch I said fuck u,0,1,1
4,0,Giving Warnings to both is not equivalent to Banning both. I am the only one blocked and I should not be considering how offensive and disgusting the other editor was to me.,1,0,0


In [73]:
subset_df.shape

(64, 5)

### Category 1: Extension Correctly Predicts Hate Speech, Baseline Incorrectly Predicts Non-Hate Speech

In [74]:
# Show full column width
pd.set_option('display.max_colwidth', None)

# Show all rows (optional)
pd.set_option('display.max_rows', None)

subset_df_cat_1 = subset_df[(subset_df['best_extension_predictions'] == 1) & (subset_df['baseline_predictions'] == 0)]
subset_df_cat_1.shape

(35, 5)

#### Pattern: Gendered/Racial Slurs

In [75]:
gender_race_pattern = r"\b(bitch|cunt|tranny|maggot|bitch face|nigger|ching chong)\b"
df_gender_race = subset_df_cat_1[subset_df_cat_1['text'].str.contains(gender_race_pattern, case=False, regex=True, na=False)].reset_index(drop = True)
df_gender_race

/var/folders/qj/tspvs3x90hg30hgcctblgwfh0000gn/T/ipykernel_42831/683372514.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_gender_race = subset_df_cat_1[subset_df_cat_1['text'].str.contains(gender_race_pattern, case=False, regex=True, na=False)].reset_index(drop = True)


,label,text,baseline_predictions,best_extension_predictions,ground_truth
0,1,Tell your bitch I said fuck u,0,1,1
1,1,"""Go change your tampon maggot!""",0,1,1
2,1,@user thank you ching chong lala \u2764\ufe0f\ud83d\udc93\u2764\ufe0f\ud83d\udc93,0,1,1
3,1,except there can never be 100% white person who has nigger ancestor. just gas them all and be done with it,0,1,1
4,1,@user ching chong wing wong,0,1,1
5,1,the ussa illegalized local tribal white justice for two reasons <number> the kk suppressed nigger savagery making the yankee left look bad <number> but travelers will be mistreated the solution to <number> is killing the tribe of benjamin not erasing local justice for all israel abraham sojourned meekly,0,1,1
6,1,Not fucking pumpkin pie? Cherry pie because it tastes fine. But pumpkin pie is the nigger of pies. And its not edible unless there is whip cream,0,1,1
7,1,"We used to do that around these part, but them those darn nazis said we couldn't throw no tranny in the river no more.",0,1,1
8,1,cute and fuck bitch face fuck her three holes spank her ass,0,1,1
9,1,"`: I do, or rather I HAVE, edited articles without an account for quite a while. My remarks were precisely as ``antagonizing`` as their remarks. The only difference is that I don't feel like a perpetual victim simply because I have a cunt. Just put out a sign saying you want radical feminists and that normal non-neurotic women need not apply. `",0,1,1


### Category 2: Extension Correctly Predicts Non-Hate Speech, Baseline Incorrectly Predicts Hate Speech

In [76]:
# Show full column width
pd.set_option('display.max_colwidth', None)

# Show all rows (optional)
pd.set_option('display.max_rows', None)

subset_df_cat_2 = subset_df[(subset_df['best_extension_predictions'] == 0) & (subset_df['baseline_predictions'] == 1)]
subset_df_cat_2.shape

(29, 5)

#### Pattern: Mentions/Placeholders

In [77]:
mentions_pattern = r"(@\w+|<user>|NEWLINE_TOKEN\w*)"
df_mentions_placeholders = subset_df_cat_2[subset_df_cat_2['text'].str.contains(mentions_pattern, case=False, regex=True, na=False)].reset_index(drop = True)
df_mentions_placeholders

/var/folders/qj/tspvs3x90hg30hgcctblgwfh0000gn/T/ipykernel_42831/2999991145.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_mentions_placeholders = subset_df_cat_2[subset_df_cat_2['text'].str.contains(mentions_pattern, case=False, regex=True, na=False)].reset_index(drop = True)


,label,text,baseline_predictions,best_extension_predictions,ground_truth
0,0,@a_man_in_black and now mods of ghazi are under fire by a mob that appears to have been led by you. :(,1,0,0
1,0,"`NEWLINE_TOKENNEWLINE_TOKENLet me get this straight, my last post, is cause for you to post something on mine saying last warning I'm going to be banned? I didn't 'accuse' you of enlisting someone else, I thought you had but you didn't it turns out, so basically by saying ``are you trying to bore me?`` that is why you are threatening to ban me? YOU KEEP POSTING ON MY TALK PAGE. STOP POSTING AND COMPLAINING AND THEN POSTING AGAIN. I'm not doing anything wrong. Take some responsibility `",1,0,0
2,0,"@user @user @user @user with the allocation of 1.127tn to justice law&amp;order. no more ""kintu kidogo""",1,0,0
3,0,"NEWLINE_TOKENWOE TO YOU CPTNONON(you did not passed, so DON'T PASS GO, DON'T COLLECT 200, GO STRAIGHT TO JAIL)",1,0,0
4,0,"@realDonaldTrump First you said these people had mental disorders. now you say kill them as soon as poss. So lets see, you want to kill people with mental illness? What if Fuck Face Eric did something similar? I could only imagine the alternative facts flying overhead! 💪😎☝",1,0,0
5,0,@user the greatest team we ever had are leaving us one after the other,1,0,0
6,0,@ArreoTheCynic A GAMER AND A WOMAN. OH LORDY.,1,0,0


### 